In [1]:
import pandas as pd 
import os, glob
import datetime
import fiona
from shapely.geometry import Point, shape, Polygon
import geopandas as gpd
import shapely.speedups
import tqdm

In [2]:
countries_df = gpd.read_file("/Users/tatiana/Documents/disaster_response/aws-disaster-response/data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp")

In [3]:
countries_df=countries_df.rename(columns={'SOV_A3':'Country Code'})

In [4]:
countries_df.head()

,featurecla,scalerank,LABELRANK,SOVEREIGNT,Country Code,ADM0_DIF,LEVEL,TYPE,ADMIN,ADM0_A3,...,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,geometry
0,Admin-0 country,0,2,Indonesia,IDN,0,2,Sovereign country,Indonesia,IDN,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4..."
1,Admin-0 country,0,3,Malaysia,MYS,0,2,Sovereign country,Malaysia,MYS,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4..."
2,Admin-0 country,0,2,Chile,CHL,0,2,Sovereign country,Chile,CHL,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611..."
3,Admin-0 country,0,3,Bolivia,BOL,0,2,Sovereign country,Bolivia,BOL,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5..."
4,Admin-0 country,0,2,Peru,PER,0,2,Sovereign country,Peru,PER,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832..."


In [5]:
df_earthquake=pd.read_csv('/Users/tatiana/Documents/disaster_response/aws-disaster-response/data/emdat_date.csv',sep=',')

In [6]:
df_earthquake.head()

,Unnamed: 0,Dis No,Year,Month,Day,Disaster Type,Disaster Subtype,Country,Country Code,Region,...,No Homeless,Total Affected,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,myDt
0,63,1960-0013-CHL,1960,5.0,22.0,Earthquake,Tsunami,Chile,CHL,South America,...,NaN,2003000.0,NaN,NaN,NaN,NaN,550000.0,4813056.0,11.427251,1960-05-22
1,64,1960-0033-DZA,1960,2.0,21.0,Earthquake,Ground movement,Algeria,DZA,Northern Africa,...,1250.0,1250.0,NaN,NaN,NaN,NaN,NaN,NaN,11.427251,1960-02-21
2,65,1961-0016-ETH,1961,6.0,2.0,Earthquake,Ground movement,Ethiopia,ETH,Eastern Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.549601,1961-06-02
3,66,1962-0044-ALB,1962,3.0,18.0,Earthquake,Ground movement,Albania,ALB,Southern Europe,...,NaN,154.0,NaN,NaN,NaN,NaN,NaN,NaN,11.688060,1962-03-18
4,67,1962-0030-COL,1962,7.0,30.0,Earthquake,Ground movement,Colombia,COL,South America,...,NaN,300.0,NaN,NaN,NaN,NaN,NaN,NaN,11.688060,1962-07-30


In [7]:
df_countries_usgs=pd.read_csv('/Users/tatiana/Documents/disaster_response/aws-disaster-response/data/merged_countries.csv',sep=',')

In [8]:
df_geo_usgs= gpd.GeoDataFrame(df_countries_usgs, geometry=gpd.points_from_xy(df_countries_usgs.longitude, df_countries_usgs.latitude))

In [9]:
df_geo_usgs.head()

,Unnamed: 0,geometry,souvereign,admin,index_right,time,latitude,longitude,depth,mag,id,place,Date
0,0,POINT (-74.46600 -8.49800),Peru,Peru,65172,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
1,13753,POINT (-74.46600 -8.49800),Peru,Peru,65172,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
2,0,POINT (-74.46600 -8.49800),Peru,Peru,1,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
3,13753,POINT (-74.46600 -8.49800),Peru,Peru,1,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
4,1,POINT (110.73600 35.51500),China,China,12,2010-01-24 00:00:00+00:00,35.515,110.736,28.7,5.0,usp000h6j0,"28 km ENE of Hancheng, China",2010-01-24


to match the emdat data set on country code - need to match the usgs data with country code

In [10]:
type(df_geo_usgs['geometry'])

geopandas.geoseries.GeoSeries

In [11]:
#getting geo fromat from usgs data into points 
points=df_geo_usgs['geometry']

In [12]:
points_list=[] #this list will contain geo points
country_code=[] #will contain country names related to the point
for p in tqdm.tqdm(points.values):
    containing_code = countries_df[countries_df['geometry'].contains(p)][['Country Code','ADM0_A3']].values
    
    if len(containing_code) > 0:
        points_list.append(p)
        country_code.append(containing_code)

100%|██████████| 17090/17090 [03:41<00:00, 77.27it/s] 


In [13]:
arr=country_code

v=[]#Country Code
j=[]#admin code
for i in arr:
    
    v.append(i.item(0))
    j.append(i.item(1))

In [14]:
d={'geometry':points_list,'Country Code':v, 'admin_code':j}
usgs_country_code=gpd.GeoDataFrame(d)

In [15]:
usgs_country_code

,geometry,Country Code,admin_code
0,POINT (-74.46600 -8.49800),PER,PER
1,POINT (-74.46600 -8.49800),PER,PER
2,POINT (-74.46600 -8.49800),PER,PER
3,POINT (-74.46600 -8.49800),PER,PER
4,POINT (110.73600 35.51500),CH1,CHN
...,...,...,...
17085,POINT (124.97800 40.16800),PRK,PRK
17086,POINT (156.67100 51.32400),RUS,RUS
17087,POINT (-100.98600 17.65500),MEX,MEX
17088,POINT (127.18700 -3.42800),IDN,IDN


In [16]:
type(usgs_country_code['geometry'])

geopandas.geoseries.GeoSeries

In [18]:
df_geo_usgs=df_geo_usgs.drop(columns=['index_right'])

In [19]:
df_country_codes = gpd.sjoin(usgs_country_code, df_geo_usgs, how="inner", op='intersects')

In [32]:
df_country_codes.head()

,geometry,Country Code,admin_code,index_right,Unnamed: 0,souvereign,admin,time,latitude,longitude,depth,mag,id,place,Date
0,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
1,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
2,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
3,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
0,POINT (-74.46600 -8.49800),PER,PER,2,0,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25


In [33]:
df_country_codes.reset_index(inplace=True)

In [34]:
df_country_codes.head()

,index,geometry,Country Code,admin_code,index_right,Unnamed: 0,souvereign,admin,time,latitude,longitude,depth,mag,id,place,Date
0,0,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
1,1,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
2,2,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
3,3,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
4,0,POINT (-74.46600 -8.49800),PER,PER,2,0,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25


In [37]:
df_country_codes.drop(columns=['index','Unnamed: 0','index_right'],inplace=True)

In [38]:
df_country_codes.head()

,geometry,Country Code,admin_code,souvereign,admin,time,latitude,longitude,depth,mag,id,place,Date
0,POINT (-74.46600 -8.49800),PER,PER,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
1,POINT (-74.46600 -8.49800),PER,PER,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
2,POINT (-74.46600 -8.49800),PER,PER,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
3,POINT (-74.46600 -8.49800),PER,PER,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
4,POINT (-74.46600 -8.49800),PER,PER,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25


In [48]:
df_country_codes.to_csv(r'/Users/tatiana/Documents/disaster_response/aws-disaster-response/data/usgs_countrycodes.csv')

In [ ]:
df_country_codes.rename(columns={Country})

In [59]:
df_test=df_country_codes.merge(df_earthquake,how='inner', left_on=['admin_code','Country Code'], right_on=['myDt','date'])

KeyError: 'date'

In [ ]:
res = df_a.merge(df_b, how='inner', left_on=['A', 'B'], right_on=['A', 'B_new'])

In [ ]:
res = df_a.merge(df_b, how='inner', left_on=['A', 'B'], right_on=['A', 'B_new'])